In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pandas_datareader as web

def get_stock_price(ticker):
  data = web.DataReader(ticker, "yahoo")
  return data['Close'].iloc[-1]

price = get_stock_price('cl=f')

price

84.25

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import lxml
from lxml import html
import requests
from collections import OrderedDict
import json

In [3]:
dtype = {'Oil (BBLS)': float, 'Water (BBLS)': float, 'Gas (MCF)': float, 'TP': float, 'CP': float, 'Comments': str}
df = pd.read_csv('data.csv', dtype=dtype)
# df = pd.read_json('data.json')
df = df.fillna('')
df.Date = pd.to_datetime(df.Date)
    
df['Gas (MCF)'] = pd.to_numeric(df['Gas (MCF)'])
df.reset_index(drop=True, inplace=True)

# Fix blanks and clip negative values
df['Oil (BBLS)'].replace('',0, inplace=True)
df['Water (BBLS)'].replace('',0, inplace=True)
df['Gas (MCF)'].replace('',0, inplace=True)
df['Oil (BBLS)'] = df['Oil (BBLS)'].clip(lower=0).round()
df['Water (BBLS)'] = df['Water (BBLS)'].clip(lower=0).round()
df['Gas (MCF)'] = df['Gas (MCF)'].clip(lower=0).round()

# Get 30 day moving average and append column to df
df = df.sort_values(['Date', 'Well Name'], ascending = [True , True])
df['7DMA'] = df.groupby('Well Name')['Oil (BBLS)'].transform(lambda x: x.rolling(7, 1).mean().round(1))
df['30DMA'] = df.groupby('Well Name')['Oil (BBLS)'].transform(lambda x: x.rolling(30, 1).mean().round(1))
df['MA_ratio'] = df['7DMA']/df['30DMA']
df = df.sort_values(['Date', 'Well Name'], ascending = [False , True])
df.head(60)

,Well Name,Date,Oil (BBLS),Gas (MCF),Water (BBLS),TP,CP,Comments,7DMA,30DMA,MA_ratio
0,Aaron #1,2021-10-12,25.0,0.0,62.0,50.0,50.0,trun on dn on vibration,75.7,67.5,1.121481
1,Aniken #1,2021-10-12,7.0,0.0,17.0,90.0,15.0,timer 12 hrs,7.4,6.0,1.233333
2,Annpick #1,2021-10-12,0.0,0.0,0.0,50.0,20.0,OFF,3.3,4.8,0.687500
3,BMMP #1,2021-10-12,10.0,646.0,0.0,400.0,40.0,,3.6,13.5,0.266667
4,Barrier #1,2021-10-12,0.0,0.0,0.0,150.0,350.0,OFF,20.1,15.1,1.331126
5,Beeler Ranch #1,2021-10-12,12.0,63.0,75.0,200.0,25.0,,17.7,27.1,0.653137
6,Bermuda #1,2021-10-12,0.0,0.0,0.0,40.0,20.0,OFF,0.0,1.5,0.000000
7,Big Sonny #1,2021-10-12,0.0,0.0,0.0,250.0,10.0,OFF,0.7,1.0,0.700000
8,Blackmon #1,2021-10-12,97.0,105.0,0.0,100.0,50.0,on,20.9,20.4,1.024510
9,Blas Reyes #1,2021-10-12,0.0,81.0,0.0,100.0,30.0,,0.0,2.1,0.000000


In [4]:
df[
    (df['Date'] == df['Date'].max()) & (~df['Comments'].str.contains('off', case=False)) & (df['30DMA'] > 4) &
(
    ((df['MA_ratio'] < 0.8) & (df['Oil (BBLS)'] < df['30DMA'] * 0.8)) | 
    (df['Oil (BBLS)'] == 0)
)
]

,Well Name,Date,Oil (BBLS),Gas (MCF),Water (BBLS),TP,CP,Comments,7DMA,30DMA,MA_ratio
3,BMMP #1,2021-10-12,10.0,646.0,0.0,400.0,40.0,,3.6,13.5,0.266667
5,Beeler Ranch #1,2021-10-12,12.0,63.0,75.0,200.0,25.0,,17.7,27.1,0.653137
12,CR #101,2021-10-12,47.0,728.0,2.0,350.0,40.0,,15.7,60.5,0.259504
14,CR #301,2021-10-12,33.0,309.0,0.0,150.0,300.0,,27.0,41.5,0.650602
18,CT #1,2021-10-12,0.0,255.0,0.0,200.0,0.0,,5.7,43.2,0.131944
23,Chad #1,2021-10-12,0.0,19.0,0.0,110.0,10.0,bad starter,4.6,13.3,0.345865
25,Circle T #1,2021-10-12,0.0,0.0,0.0,50.0,25.0,turn on,8.6,10.5,0.819048
26,Cross S #1,2021-10-12,0.0,0.0,0.0,140.0,30.0,trun on,3.1,6.0,0.516667
42,Joe #1,2021-10-12,0.0,0.0,52.0,40.0,20.0,,1.4,6.6,0.212121
43,Joseph #1,2021-10-12,2.0,0.0,10.0,90.0,40.0,,3.3,5.1,0.647059
